In [ ]:
# Import necessary libraries below
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns 
sns.set(style='darkgrid')
sns.set_context('poster')

from datetime import datetime
from scipy import stats
from sklearn.linear_model import LogisticRegression

import folium
from folium import plugins
from folium.plugins import HeatMap
from folium.plugins import MarkerCluster

In [ ]:
# import properties_2016 df
df_properties = pd.read_csv('properties_2016.csv')
df_properties.head()

In [ ]:
# import train_2016 df
df_train = pd.read_csv('train_2016_v2.csv')
df_train.head()

In [ ]:
# check shape 
print(df_properties.shape)
print(df_train.shape)

In [ ]:
# merge df and df_train
df = pd.merge(df_train, df_properties, on='parcelid')
df.head()

In [ ]:
## rearrange df
#df.tail().transpose()

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
# correlation heatmap
plt.figure(figsize=(30, 20))
sns.heatmap(data = df.corr())
plt.show()

In [ ]:
# create df for missing values
# percentage of NaN values
df_missing = df.isna().mean().round(4) * 100
df_missing

In [ ]:
# prepare to plot df_missing
df_missing = df.isna().sum(axis=0).reset_index()
df_missing.columns = ['column_name', 'missing_count']
df_missing = df_missing.loc[df_missing['missing_count']>0]
df_missing = df_missing.sort_values(by='missing_count')

In [ ]:
# plot df_missing
fig, ax = plt.subplots(figsize=(20, 30))
sns.barplot(x='missing_count', y='column_name', data = df_missing)
ax.set_xlabel('Count of NaN values')
ax.set_title('Number of NaN values in each column')

In [ ]:
# plot with different color palette
fig, ax = plt.subplots(figsize=(20, 30))
sns.barplot(x='missing_count', y='column_name', palette='ch:.25', data = df_missing)
ax.set_xlabel('Count of NaN values')
ax.set_title('Number of NaN values in each column')

In [ ]:
df_missing = (df.isnull().sum()/len(df)).reset_index()
df_missing.columns = ['column','proportion']
df_missing = df_missing.sort_values(by = 'proportion')
print(df_missing)
missingvaluescols = df_missing[df_missing['proportion'] > 0.50].column.tolist()
df = df.drop(missingvaluescols, axis=1)
df.head()

In [ ]:
df.dtypes

In [ ]:
df_float = df.drop(['parcelid','transactiondate', 'propertycountylandusecode', 'propertyzoningdesc'], axis=1)
df_float.dtypes

In [ ]:
# correlation heatmap
sns.set_context('poster')
plt.figure(figsize=(40, 30))
sns.heatmap(data = df_float.corr())
plt.show()

In [ ]:
# check duplicates
duplicates = df[df.duplicated()]
print(len(duplicates))
duplicates.head()

In [ ]:
# plot logerror
plt.figure(figsize=(20,10))
sns.distplot(df.logerror.values, bins=50, color='magenta', kde=False)
plt.xlabel('logerror')
plt.show()

In [ ]:
df.columns

In [ ]:
# convert transactiondate to datetime
df.transactiondate = pd.to_datetime(df.transactiondate)

In [ ]:
# plot transactions by month
df['transaction_month'] = df['transactiondate'].dt.month
month = df['transaction_month'].value_counts()
plt.figure(figsize=(20, 10))
sns.barplot(month.index, month.values, alpha=0.8)
plt.xticks(rotation='vertical')
plt.xlabel('Month of transaction')
plt.ylabel('Number of transactions')
plt.show()

In [ ]:
df_type = df.dtypes.reset_index()
df_type.columns=['Count', 'Column Type']
df_type.head()

In [ ]:
df_type.groupby('Column Type').aggregate('count').reset_index()

In [ ]:
df.latitude.head()

In [ ]:
df.longitude.head()

In [ ]:
# check if we have a valid lat and long (choose the first row)
new_map = df
len(df)
new_map = folium.Map(location=[34.2809, -118.488],
                        zoom_start=13,
                        tiles='openstreetmap')
new_map

In [ ]:
# create df for mapping
df_geo = df[['latitude', 'longitude', 'logerror']]
df_geo.head()

In [ ]:
df.propertylandusetypeid.unique()

In [ ]:
len(df.propertylandusetypeid)

In [ ]:
print(df_properties.architecturalstyletypeid.unique())

df_properties.loc[df_properties['architecturalstyletypeid'] == 2]['yearbuilt']

In [ ]:
# the zillow data dictionary says the have "Latitude of the middle of the parcel 
# multiplied by 10e6", so we have to normalize it to be readable by Folium
df_geo['longitude']/=1e6
df_geo['latitude']/=1e6

In [ ]:
# normalized latitude
df_geo.latitude.head()

In [ ]:
# normalized longitude
df.longitude.head()

In [ ]:
# replace the lat and lonf in the merged df
df['latitude'] = df_geo.latitude
df['longitude'] = df_geo.longitude

In [ ]:
# new_map = df
# len(df)
# new_map = folium.Map(location=[34.2809, -118.488],
#                         zoom_start=13,
#                         tiles='openstreetmap')
# new_map

# for i in df.index:
#     lat = df.latitude[i]
#     long = df.longitude[i]
#     logerror = df.logerror[i]
#     marker = folium.Marker([lat, long]).add_to(new_map)
#     marker = folium.Marker([lat, long]).add_to(new_map)

# new_map

In [ ]:
df_geo.dropna(subset=['latitude', 'longitude'], axis=0, inplace=True)
df_geo.head()

In [ ]:
plt.figure(figsize=(20,10))
ulimit = np.percentile(df.logerror.values, 99)
llimit = np.percentile(df.logerror.values, 1)
df['logerror'].loc[df['logerror']>ulimit] = ulimit
df['logerror'].loc[df['logerror']<llimit] = llimit

plt.figure(figsize=(12,8))
sns.distplot(df.logerror.values, bins=50, color='cyan', kde=False)
plt.xlabel('logerror')
plt.show()

In [ ]:

sns.set_palette(sns.color_palette("cubehelix", 8))
sns.set_context('paper')

plt.figure(figsize=(12,12))
sns.jointplot(df.latitude.values, df.longitude.values, size=10)
plt.ylabel('Longitude', fontsize=12)
plt.xlabel('Latitude', fontsize=12)
plt.show()


In [ ]:
plt.figure(figsize=(12,12))
sns.set_context('paper')
cmap = sns.cubehelix_palette(light=1, as_cmap=True)
sns.kdeplot(df.latitude.values, df.longitude.values, cmap=cmap, shade=True);

# pal = sns.dark_palette("palegreen", as_cmap=True)
# sns.kdeplot(x, y, cmap=pal);

In [ ]:
# create map of every house
sns.set_palette(sns.color_palette("cubehelix"))
sns.set_context('paper')

plt.figure(figsize=(12,12))
sns.scatterplot(df.latitude.values, df.longitude.values)
plt.ylabel('Longitude', fontsize=12)
plt.xlabel('Latitude', fontsize=12)
plt.show()

In [ ]:
unique = df.yearbuilt.unique()
sorted_unique = sorted(unique)
print(min(sorted_unique))
print(max(sorted_unique))

In [ ]:
# yearbuilt frequency barplot
df['yearbuilt'].plot.hist(range=(1885, 2015), bins=20, color='darkblue')
plt.title('Year bins', color = 'orange')
plt.show()

In [ ]:
# make new column for year_built with 3 bins
# 1 before 1900
# 2 between 1900 and 1940
# 3 between 1940 and 1970
# 4 between 1970 and 2000
# 5 after 2000

df['year_bins'] = df.yearbuilt.map(lambda x: 1 if x >= 1885 and x <= 1900
                            else (2 if x > 1900 and x <= 1940
                                  else (3 if x > 1940 and x <= 1970
                                       else (4 if x > 1970 and x <= 2000
                                            else 5))))
df.head(5)

In [ ]:
df.year_bins.unique()

In [ ]:
# create map of every house by year of built
sns.set_palette(sns.color_palette("cubehelix"))
sns.set_context('paper')
sns.set(style='whitegrid')

# '2000-2015'
lat5 = df.loc[df['year_bins'] == 5].latitude
long5 = df.loc[df['year_bins'] == 5].longitude
# '1970-2000'
lat4 = df.loc[df['year_bins'] == 4].latitude
long4 = df.loc[df['year_bins'] == 4].longitude
# '1940-1970'
lat3 = df.loc[df['year_bins'] == 3].latitude
long3 = df.loc[df['year_bins'] == 3].longitude
# '1900-1940'
lat2 = df.loc[df['year_bins'] == 2].latitude
long2 = df.loc[df['year_bins'] == 2].longitude
# '1885-1900'
lat1 = df.loc[df['year_bins'] == 1].latitude
long1 = df.loc[df['year_bins'] == 1].longitude

plt.figure(figsize=(12,12))
sns.scatterplot(lat5.values, long5.values, label = '2000-2015', color='orange')
sns.scatterplot(lat4.values, long4.values, label = '1970-2000', color='red')
sns.scatterplot(lat3.values, long3.values, label = '1940-1970', color='plum')
sns.scatterplot(lat2.values, long2.values, label = '1900-1940', color='teal')
sns.scatterplot(lat1.values, long1.values, label = '1885-1900', color='navy')

plt.title('California houses by year of built', color = 'navy')
plt.legend()

plt.ylabel('Longitude')
plt.xlabel('Latitude')
plt.show()

In [ ]:
fig = plt.figure(figsize = (24,14))

# ax1 = fig.add_subplot(221)
# sns.scatterplot(lat1.values, long1.values, label = '1885-1900', color='navy')
# plt.legend()

ax1 = fig.add_subplot(221)
plt.scatter(long1, lat1, label = '1885-1900', color='darkblue', marker='.')
plt.title('Houses built from 1885 to 1900')

ax2 = fig.add_subplot(222)
sns.scatterplot(lat2.values, long2.values, label = '1900-1940', color='teal')
plt.title('Houses built from 1900 to 1940')

ax3 = fig.add_subplot(223)
sns.scatterplot(lat3.values, long3.values, label = '1940-1970', color='plum')
plt.title('Houses built from 1940 to 1970')

ax4 = fig.add_subplot(224)
sns.scatterplot(lat4.values, long4.values, label = '1970-2000', color='red')
plt.title('Houses built from 1970 to 2000')

# ax5 = fig.add_subplot(225)
# sns.scatterplot(lat5.values, long5.values, label = '2000-2015', color='orange')
# plt.title('Houses built from 2000 to 2015')

plt.ylabel('Longitude')
plt.xlabel('Latitude')
plt.show()

In [ ]:

fig = plt.figure(figsize = (7,4.5))

sns.scatterplot(lat5.values, long5.values, label = '2000-2015', color='orange')
plt.title('Houses built from 2000 to 2015')

In [ ]:

plt.figure(figsize=(12,12))
sns.set_context('paper')
cmap = sns.cubehelix_palette(light=1, as_cmap=True)

z = df.logerror
plt.scatter(df.latitude.values, df.longitude.values, s=z*2000, c=x, cmap=cmap, alpha=0.4, linewidth=2)
# plt.scatter(x, y, s=z*2000, c=x, cmap="Blues", alpha=0.4, edgecolors="grey", linewidth=2)

plt.show()

In [ ]:

fig = plt.figure(figsize = (24,14))
ax1= fig.add_subplot(222)

df['yearbuilt'] = df['yearbuilt'].map(lambda x:str(x).split('.')[0])
year = df.groupby(['yearbuilt', 'roomcnt'])['parcelid'].count().unstack('roomcnt').fillna(0)
year.plot(kind='bar', stacked=True,ax=ax1)

In [ ]:
df.describe()

In [ ]:
# logerror 75%
logerror_75= df['logerror'].quantile(0.75)

# make a dataframe with the quantile
big_logerror = df[(df['logerror'] >= logerror_75)]
big_logerror.head()

In [ ]:
len(big_logerror)